In [1]:
import pyspark
from pyspark import SparkContext
sc =SparkContext()

In [2]:
nums= sc.parallelize([1,2,3,4])


In [3]:
nums.take(1)


[1]

In [4]:
squared = nums.map(lambda x: x*x).collect()
for num in squared:
    print('%i ' % (num))

1 
4 
9 
16 


In [5]:
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
# create the list of tuple with the information
list_p = [('John',19),('Smith',29),('Adam',35),('Henry',50)]

# 2. Build the RDD
rdd = sc.parallelize(list_p)

# 3. Convert to tuples
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))

# 4. Create a dataframe context
DF_ppl = sqlContext.createDataFrame(ppl)

In [7]:
DF_ppl.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



## PySpark Machine Learning

In [8]:
# Step 1: Basic Operation with PySpark
import pandas as pd
url = "https://raw.githubusercontent.com/sadhana1002/PredictingSalaryClass-Classification/master/adult.csv"
df = sqlContext.createDataFrame(pd.read_csv(url,
                                            names=['Age','workclass',
                                                   'fnlwgt','education',
                                                   'education_num',
                                                   'marital',
                                                   'occupation',
                                                   'relationship',
                                                   'race', 'sex',
                                                   'capital_gain',
                                                   'capital_loss',
                                                   'hours_week',
                                                   'native_country',
                                                   'label']))

df.printSchema()

root
 |-- Age: long (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: long (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: long (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: long (nullable = true)
 |-- capital_loss: long (nullable = true)
 |-- hours_week: long (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)



In [9]:
df.show(5,truncate=False)

+---+-----------------+------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+----------+--------------+------+
|Age|workclass        |fnlwgt|education |education_num|marital            |occupation        |relationship  |race  |sex    |capital_gain|capital_loss|hours_week|native_country|label |
+---+-----------------+------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+----------+--------------+------+
|39 | State-gov       |77516 | Bachelors|13           | Never-married     | Adm-clerical     | Not-in-family| White| Male  |2174        |0           |40        | United-States| <=50K|
|50 | Self-emp-not-inc|83311 | Bachelors|13           | Married-civ-spouse| Exec-managerial  | Husband      | White| Male  |0           |0           |13        | United-States| <=50K|
|38 | Private         |215646| HS-grad  |9            | Divorced          | Hand

In [10]:
# import all from `sql.types`
from pyspark.sql.types import *

# wrie a custom function to convert the data type of DataFrame columns
def convertColumn(df,names,newType):
    for name in names:
        df = df.withColumn(name,df[name].cast(newType))
    return df

#list of continuous features
CONTINUOUS_FEATURES=['age','fnlwgt','capital_gain','education_num', 'capital_loss', 'hours_week']

# convert the type
df = convertColumn(df,CONTINUOUS_FEATURES, FloatType())

df.printSchema()

root
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: float (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: float (nullable = true)
 |-- capital_loss: float (nullable = true)
 |-- hours_week: float (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)



In [11]:
df.select('age','fnlwgt').show(5)

+----+--------+
| age|  fnlwgt|
+----+--------+
|39.0| 77516.0|
|50.0| 83311.0|
|38.0|215646.0|
|53.0|234721.0|
|28.0|338409.0|
+----+--------+
only showing top 5 rows



In [12]:
df.groupBy("education").count().sort("count",ascending=True).show()

+-------------+-----+
|    education|count|
+-------------+-----+
|    Preschool|   51|
|      1st-4th|  168|
|      5th-6th|  333|
|    Doctorate|  413|
|         12th|  433|
|          9th|  514|
|  Prof-school|  576|
|      7th-8th|  646|
|         10th|  933|
|   Assoc-acdm| 1067|
|         11th| 1175|
|    Assoc-voc| 1382|
|      Masters| 1723|
|    Bachelors| 5355|
| Some-college| 7291|
|      HS-grad|10501|
+-------------+-----+



In [13]:
df.describe().show()
df.describe('capital_gain').show()

+-------+------------------+------------+------------------+-------------+------------------+---------+-----------------+------------+-------------------+-------+------------------+------------------+------------------+--------------+------+
|summary|               age|   workclass|            fnlwgt|    education|     education_num|  marital|       occupation|relationship|               race|    sex|      capital_gain|      capital_loss|        hours_week|native_country| label|
+-------+------------------+------------+------------------+-------------+------------------+---------+-----------------+------------+-------------------+-------+------------------+------------------+------------------+--------------+------+
|  count|             32561|       32561|             32561|        32561|             32561|    32561|            32561|       32561|              32561|  32561|             32561|             32561|             32561|         32561| 32561|
|   mean| 38.58164675532078|    

In [14]:
## Crosstab computation
df.crosstab('age','label').sort("age_label").show()

+---------+------+-----+
|age_label| <=50K| >50K|
+---------+------+-----+
|     17.0|   395|    0|
|     18.0|   550|    0|
|     19.0|   710|    2|
|     20.0|   753|    0|
|     21.0|   717|    3|
|     22.0|   752|   13|
|     23.0|   865|   12|
|     24.0|   767|   31|
|     25.0|   788|   53|
|     26.0|   722|   63|
|     27.0|   754|   81|
|     28.0|   748|  119|
|     29.0|   679|  134|
|     30.0|   690|  171|
|     31.0|   705|  183|
|     32.0|   639|  189|
|     33.0|   684|  191|
|     34.0|   643|  243|
|     35.0|   659|  217|
|     36.0|   635|  263|
+---------+------+-----+
only showing top 20 rows



In [15]:
df.drop('education_num').columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'marital',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_week',
 'native_country',
 'label']

In [16]:
# filter data
df.filter(df.age>40).count()

13443

In [17]:
# decriptive statistics by group
df.groupby('marital').agg({'capital_gain':'mean'}).show()

+--------------------+------------------+
|             marital| avg(capital_gain)|
+--------------------+------------------+
|             Widowed| 571.0715005035247|
| Married-spouse-a...| 653.9832535885167|
|   Married-AF-spouse| 432.6521739130435|
|  Married-civ-spouse|1764.8595085470085|
|            Divorced| 728.4148098131893|
|       Never-married|376.58831788823363|
|           Separated| 535.5687804878049|
+--------------------+------------------+



## Step 2: Data Preprocessing

In [18]:
# Add age square
from pyspark.sql.functions import *

# 1 select the column
age_square = df.select(col("age")**2)

# 2 apply the transformation and add it to the DataFrame
df = df.withColumn("age_square",col("age")**2)

df.printSchema()

root
 |-- age: float (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: float (nullable = true)
 |-- marital: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: float (nullable = true)
 |-- capital_loss: float (nullable = true)
 |-- hours_week: float (nullable = true)
 |-- native_country: string (nullable = true)
 |-- label: string (nullable = true)
 |-- age_square: double (nullable = true)



In [19]:
COLUMNS = ['age', 'age_square', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital',
           'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss',
           'hours_week', 'native_country', 'label']
df = df.select(COLUMNS)
df.first()

Row(age=39.0, age_square=1521.0, workclass=' State-gov', fnlwgt=77516.0, education=' Bachelors', education_num=13.0, marital=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', sex=' Male', capital_gain=2174.0, capital_loss=0.0, hours_week=40.0, native_country=' United-States', label=' <=50K')

In [20]:
df.filter(df.native_country == 'Holand-Netherlands').count()
df.groupby('native_country').agg({'native_country': 'count'}).sort(asc("count(native_country)")).show()
df_remove = df.filter(df.native_country != 'Holand-Netherlands')


+--------------------+---------------------+
|      native_country|count(native_country)|
+--------------------+---------------------+
|  Holand-Netherlands|                    1|
|            Scotland|                   12|
|            Honduras|                   13|
|             Hungary|                   13|
| Outlying-US(Guam...|                   14|
|          Yugoslavia|                   16|
|                Laos|                   18|
|            Thailand|                   18|
|            Cambodia|                   19|
|     Trinadad&Tobago|                   19|
|                Hong|                   20|
|             Ireland|                   24|
|             Ecuador|                   28|
|              France|                   29|
|              Greece|                   29|
|                Peru|                   31|
|           Nicaragua|                   34|
|            Portugal|                   37|
|                Iran|                   43|
|         

In [21]:
## Pyspark Pipeline API
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

stringIndexer = StringIndexer(inputCol="workclass", outputCol="workclass_encoded")
model = stringIndexer.fit(df)
indexed= model.transform(df)
encoder = OneHotEncoder(dropLast=False,inputCol="workclass_encoded",outputCol="workclass_vec").fit(indexed)
encoded = encoder.transform(indexed)
encoded.show(2)

+----+----------+-----------------+-------+----------+-------------+-------------------+----------------+--------------+------+-----+------------+------------+----------+--------------+------+-----------------+-------------+
| age|age_square|        workclass| fnlwgt| education|education_num|            marital|      occupation|  relationship|  race|  sex|capital_gain|capital_loss|hours_week|native_country| label|workclass_encoded|workclass_vec|
+----+----------+-----------------+-------+----------+-------------+-------------------+----------------+--------------+------+-----+------------+------------+----------+--------------+------+-----------------+-------------+
|39.0|    1521.0|        State-gov|77516.0| Bachelors|         13.0|      Never-married|    Adm-clerical| Not-in-family| White| Male|      2174.0|         0.0|      40.0| United-States| <=50K|              4.0|(9,[4],[1.0])|
|50.0|    2500.0| Self-emp-not-inc|83311.0| Bachelors|         13.0| Married-civ-spouse| Exec-manage

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
CATE_FEATURES = ['workclass','education','marital','occupation','relationship','race','sex','native_country']

stages=[]
for categoricalCol in CATE_FEATURES:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()],
                            outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [23]:
# 1. Index the label feature
label_stringIdx = StringIndexer(inputCol="label",outputCol="newlabel")
stages += [label_stringIdx]

In [24]:
assemblerInputs = [c + "classVec" for c in CATE_FEATURES] + CONTINUOUS_FEATURES


In [25]:
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [26]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df_remove)
model = pipelineModel.transform(df_remove)

In [27]:
model.take(1)


[Row(age=39.0, age_square=1521.0, workclass=' State-gov', fnlwgt=77516.0, education=' Bachelors', education_num=13.0, marital=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', sex=' Male', capital_gain=2174.0, capital_loss=0.0, hours_week=40.0, native_country=' United-States', label=' <=50K', workclassIndex=4.0, workclassclassVec=SparseVector(8, {4: 1.0}), educationIndex=2.0, educationclassVec=SparseVector(15, {2: 1.0}), maritalIndex=1.0, maritalclassVec=SparseVector(6, {1: 1.0}), occupationIndex=3.0, occupationclassVec=SparseVector(14, {3: 1.0}), relationshipIndex=1.0, relationshipclassVec=SparseVector(5, {1: 1.0}), raceIndex=0.0, raceclassVec=SparseVector(4, {0: 1.0}), sexIndex=0.0, sexclassVec=SparseVector(1, {0: 1.0}), native_countryIndex=0.0, native_countryclassVec=SparseVector(41, {0: 1.0}), newlabel=0.0, features=SparseVector(100, {4: 1.0, 10: 1.0, 24: 1.0, 32: 1.0, 44: 1.0, 48: 1.0, 52: 1.0, 53: 1.0, 94: 39.0, 95: 77516.0, 96: 2174.0, 9

In [28]:
from pyspark.ml.linalg import DenseVector
input_data = model.rdd.map(lambda x: (x["newlabel"], DenseVector(x["features"])))

In [29]:
df_train = sqlContext.createDataFrame(input_data, ["label","features"])
df_train.show(2)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,0.0,0.0,0.0,...|
|  0.0|[0.0,1.0,0.0,0.0,...|
+-----+--------------------+
only showing top 2 rows



In [30]:
# Create a train/test set
# split the data into train and test sets
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)



In [31]:
train_data, test_data = df_train.randomSplit([.8,.2],seed=1234)

train_data.groupby('label').agg({'label': 'count'}).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|       19765|
|  1.0|        6332|
+-----+------------+



In [32]:
# Build the logistic Regressor

from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="label", featuresCol="features",
                        maxIter=10, regParam=0.3)

# fit the data to the model
linearModel = lr.fit(train_data)

In [33]:
# print the coefficients and intercepts for logistic regression
print("Coefficients: " + str(linearModel.coefficients))
print("Intercept: " + str(linearModel.intercept))

Coefficients: [-0.027290577453279757,-0.056307005393697075,0.024014714263646774,-0.1498986581079109,-0.023220913603591506,0.31103189416435734,0.2465355266831762,-0.4320323201821748,-0.14201674880411516,-0.045060383957739196,0.22731267672590738,0.3665643557623941,0.04918258486519807,-0.21121733648236754,-0.0014939916520399616,-0.21401450345593062,-0.270450053207931,0.5076203111739022,-0.2533202064173693,-0.16422410448664918,0.5243341394763534,-0.2149775741255282,-0.2237353211211303,0.4045061135975721,-0.3132698666084187,-0.1637293558925834,-0.19284789351309475,-0.15668054113922752,-0.18220085233976593,0.2430591751689406,-0.030567838181009728,0.33786393849610624,-0.08582938615105766,0.06493140886035424,-0.2507840316264174,-0.18099716010960168,-0.1505303851412841,-0.07195124275246355,-0.250315824662858,-0.28791651672632024,0.1487166422042849,0.13979938648326523,-0.2609242243227046,0.33514643455197457,-0.1852504892789734,-0.28924477482011823,-0.21317474367597594,0.4515600027477449,0.087460

In [34]:
# make predictions on test data using the transform method
predictions = linearModel.transform(test_data)
predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [35]:
selected = predictions.select("label", "prediction", "probability")
selected.show(20)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  0.0|       0.0|[0.71110183486555...|
|  0.0|       0.0|[0.81137768456323...|
|  0.0|       0.0|[0.83282104994684...|
|  0.0|       0.0|[0.87398722120359...|
|  0.0|       0.0|[0.86356901524802...|
|  0.0|       0.0|[0.84747267118820...|
|  0.0|       0.0|[0.88893577786996...|
|  0.0|       0.0|[0.62937281238927...|
|  0.0|       0.0|[0.79150023963191...|
|  0.0|       1.0|[0.42670421233883...|
|  0.0|       1.0|[0.46672738477910...|
|  0.0|       0.0|[0.74175412695004...|
|  0.0|       0.0|[0.66975987122783...|
|  0.0|       1.0|[0.48303074427070...|
|  0.0|       0.0|[0.60799223556090...|
|  0.0|       0.0|[0.72789452303959...|
|  0.0|       1.0|[0.46106511653089...|
|  0.0|       0.0|[0.86445344690623...|
|  0.0|       0.0|[0.82605995297226...|
|  0.0|       0.0|[0.91910915855547...|
+-----+----------+--------------------+
only showing top 20 rows



In [36]:
cm = predictions.select("label", "prediction")


In [37]:
cm.groupby('label').agg({'label': 'count'}).show()


+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|        4955|
|  1.0|        1509|
+-----+------------+



In [38]:
cm.groupby('prediction').agg({'prediction': 'count'}).show()


+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|       0.0|             5814|
|       1.0|              650|
+----------+-----------------+



In [39]:
cm.filter(cm.label == cm.prediction).count() / cm.count()


0.8296720297029703

In [40]:
def accuracy_m(model): 
    predictions = model.transform(test_data)
    cm = predictions.select("label", "prediction")
    acc = cm.filter(cm.label == cm.prediction).count() / cm.count()
    print("Model accuracy: %.3f%%" % (acc * 100)) 
accuracy_m(model = linearModel)

Model accuracy: 82.967%


## ROC Metrics
Binary Classification Evaluator module includes ROC measures.

In [41]:
### Use Roc
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#evaulate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print(evaluator.evaluate(predictions))
print(evaluator.getMetricName())

0.8916416068005021
areaUnderROC


## Tune the Hyperparameter
Grid search can be done, just like in sklearn

In [42]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam,[0.01,0.5])
             .build())

In [43]:
from time import *
start_time = time()

# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(train_data)
# likely take a fair amount of time
end_time = time()
elapsed_time = end_time - start_time
print("Time to train model: %.3f seconds" % elapsed_time)


Time to train model: 53.358 seconds


In [44]:
accuracy_m(model = cvModel)


Model accuracy: 85.179%


In [45]:
bestModel = cvModel.bestModel
bestModel.extractParamMap()

{Param(parent='LogisticRegression_6b20ed6bbcd2', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto',
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_6b20ed6bbcd2', name='maxBlockSizeInMB', doc='maximum memory in MB for s